In [1]:
using CairoMakie, QuantEcon, LinearAlgebra, Optim, Dierckx, QuadGK

In [18]:
include("crra_code.jl")
include("ces_code.jl")

hpi_CES (generic function with 2 methods)

In [11]:
pars_crra = (;A = 0.01, # total factor productivity
    α = 0.33, # capital share
    δ = 0.08, # depreciation rate
    r = 0.00, # interest rate
    w = 1.0, # wage
    β = 0.9, # discount factor
    γ = 1.0, # risk aversion
    nz = 19, # number of grid points for z
    ρ = 0.9, # persistence of AR1
    μ = 0.0, # mean of AR1
    σ = 0.003, # std of AR1
    na = 101, # number of asset grid points
    ϕ = 0.0, # borrowing constraint
    θ = 4.0, # grid expansion factor
    toler = 4e-7, # tolerance
    maxiter = 1000, # maximum no. of iterations
    print_skip = 1000, # how often to print
    how_iter = 50, # number of Howard iterations
    toler_prices = 1e-3, # tolerance
    maxiter_prices = 100, # maximum no. of iterations
    print_skip_prices = 1, # how often to print
    lb = 0.0, # lower bound of capital grid
    ub = 10000.0) # upper bound of capital grid

(A = 0.01, α = 0.33, δ = 0.08, r = 0.0, w = 1.0, β = 0.9, γ = 1.0, nz = 19, ρ = 0.9, μ = 0.0, σ = 0.003, na = 101, ϕ = 0.0, θ = 4.0, toler = 4.0e-7, maxiter = 1000, print_skip = 1000, how_iter = 50, toler_prices = 0.001, maxiter_prices = 100, print_skip_prices = 1, lb = 0.0, ub = 10000.0)

In [12]:
function invariant_distribution(M, O, X, Y, Inv, policy, Π, Avals, n, pars)
    (; na, nz) = pars
    for i in 1:na
        for j in 1:nz
            if policy[i,j] <+ Avals[1]
                M[i,j,1,:] = Π[j,:]
            elseif policy[i,j] >= Avals[end]
                M[i,j,end,:] = Π[j,:]
            else
                index = findfirst(x -> x > policy[i,j], Avals)
                π = (Avals[index] - policy[i,j]) / (Avals[index] - Avals[index-1])
                M[i,j,index-1,:] = π * Π[j,:]
                M[i,j,index,:] = (1-π) * Π[j,:]
            end
        end
    end
    O = reshape(M, n, n)
    for i in 1:na
        O[i,:] = O[i,:] / sum(O[i,:])
    end
    X = reduce(vcat, [I(n) - O', ones(1,n)])
    Y = [zeros(n);1]
    Inv = X \ Y
    Inv[Inv .< 0] .= 0
    Inv = Inv / sum(Inv)
    return Inv
end

invariant_distribution (generic function with 1 method)

In [13]:
function labour_supply(Π, Zvals, pars)
    (; nz) = pars
    L = zeros(nz)
    L2 = reduce(vcat, [I(nz) - Π', ones(1,nz)])
    L3 = [zeros(nz);1]
    L = L2 \ L3
    L[L .< 0] .= 0
    L = L / sum(L)
    L = L' * Zvals
    return L
end

labour_supply (generic function with 1 method)

In [14]:
function equilibrium_vfi_crra(pars)
    (; na, nz, toler_prices, maxiter_prices, print_skip_prices, r, w, β, A, α, ϕ, δ) = pars
    n = na * nz
    M = zeros(na, nz, na, nz)
    O = zeros(n, n)
    X = zeros(n+1,n)
    Y = zeros(n+1)
    Inv = zeros(n)
    Π, Zvals = ar1(pars)
    Avals = exp_grid(pars)
    v_init = zeros(na,nz)
    policy = similar(v_init)
    wealth = zeros(na,nz)
    initial_distribution = zeros(n,1)
    L = labour_supply(Π, Zvals, pars)
    Ks, Kd = 1, 1
    iter = 0
    error = toler_prices + 1
    rnew = 1/β - 1
    if iter == 0
        println("Iterating on prices...")
    end
    while ((error > toler_prices) && (iter < maxiter_prices))
        println("////////////////////")
        println("Price Iteration: $iter")
        r_iter = (r + rnew) / 2
        Kd = ((A^α * L ^ (1-α)) / (r_iter + δ))^1/(1-α)
        w_iter = (1-α) * A * (Kd/L)^α
        Φ = w_iter * (exp(minimum(Zvals))/r_iter)
        if ϕ > 0
            ϕ_iter = min(Φ, exp(minimum(Zvals)))
            pars = (; pars..., ϕ = ϕ_iter, r = r_iter, w = w_iter)
        else
            pars = (; pars..., r = r_iter, w = w_iter)
        end
        v_new, policy = vfi(v_init, policy, Π, Zvals, Avals, pars)
        Invariant = invariant_distribution(M, O, X, Y, Inv, policy, Π, Avals, n, pars)
        G = reshape(policy .- ϕ, n, 1)
        Ks = dot(Invariant', G)
        diff = Ks - Kd
        error = abs(diff)
        if diff > 0
            rnew = r_iter
        else
            r = r_iter
        end
        if iter % print_skip_prices == 0
            println("%%%%%%%%%%%%%%%%%%%%")
            println("Iter = $iter, Ks = $Ks, Kd = $Kd, diff = $diff")
            println("%%%%%%%%%%%%%%%%%%%%")
        end
        v_init .= v_new
        iter += 1
    end
    r_final = r + rnew / 2
    w_final = (1-α) * A * (Kd/L)^α
    Invariant = reshape(Invariant, na, nz)
    for i in 1:na
        for j in 1:nz
            wealth[i,j] = w_final * exp(Zvals[j]) + (1 + r_final) * Avals[i]
        end
    end
    println("%%%%%%%%%%%%%%%%%%%%")
    println("r = $r_final, w = $w_final")
    println("%%%%%%%%%%%%%%%%%%%%")
    return v_init, policy, Invariant, wealth
end

@time begin 
    v_vfi_crra, pol_vfi_crra, dist_vfi_crra, wealth_vfi_crra = equilibrium_vfi_crra(pars_crra)
end

Iterating on prices...
////////////////////
Price Iteration: 0
Iterating...
--------------------
Iteration: 0, Error: 0.9025766576759272
--------------------
Converged in 150 iterations for r = 0.05555555555555558 and w = 0.016858265807657185
--------------------
%%%%%%%%%%%%%%%%%%%%
Iter = 0, Ks = 0.04493696684536476, Kd = 0.049144974801661453, diff = -0.004208007956296694
%%%%%%%%%%%%%%%%%%%%
////////////////////
Price Iteration: 1
Iterating...
--------------------
Iteration: 0, Error: 0.15290645633480163
--------------------
Converged in 118 iterations for r = 0.08333333333333337 and w = 0.015852469300602034
--------------------
%%%%%%%%%%%%%%%%%%%%
Iter = 1, Ks = 0.0592194863861838, Kd = 0.04078698588981427, diff = 0.018432500496369533
%%%%%%%%%%%%%%%%%%%%
////////////////////
Price Iteration: 2
Iterating...
--------------------
Iteration: 0, Error: 0.08118289553258673
--------------------
Converged in 101 iterations for r = 0.06944444444444448 and w = 0.016324251888359266
--------

([-42.679925646639525 -42.255661577927725 … -28.5430635477603 -25.838432234649783; -42.4258996685011 -42.08236993502144 … -28.54179737033793 -25.837484527129508; … ; 65.09239548107321 65.09239567129121 … 65.09288328862324 65.09314125851665; 65.49439379656458 65.49439397928543 … 65.49486237845404 65.49511018148867], [2.2682167995095997e-16 4.240185886515615e-16 … 0.4136870039015336 0.6484236914717035; 3.5663618627803524e-16 2.822037545510818e-16 … 0.4137820237114716 0.6485190182944899; … ; 9170.648659816043 9170.648834262916 … 9171.096063236859 9171.332680291345; 9546.82165702209 9546.821831458736 … 9547.268988744003 9547.505560222056], [8.961454588941165e-8 2.8743191231633174e-6 … 2.872944900944026e-6 8.852717079388527e-8; 4.5203240671871035e-8 2.074915631354517e-6 … 2.0735834531834728e-6 4.411566437101412e-8; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0003667857771892585 0.0005606192817358414 … 0.497458359385241 0.7603477710318676; 0.00047598716607814736 0.0006698206706247303 … 0.4

In [15]:
function equilibrium_hpi_crra(pars)
    (; na, nz, toler_prices, maxiter_prices, print_skip_prices, r, w, β, A, α, ϕ, δ) = pars
    n = na * nz
    M = zeros(na, nz, na, nz)
    O = zeros(n, n)
    X = zeros(n+1,n)
    Y = zeros(n+1)
    Inv = zeros(n)
    Π, Zvals = ar1(pars)
    Avals = exp_grid(pars)
    v_init = zeros(na,nz)
    policy = similar(v_init)
    wealth = zeros(na,nz)
    initial_distribution = zeros(n,1)
    L = labour_supply(Π, Zvals, pars)
    Ks, Kd = 1, 1
    iter = 0
    error = toler_prices + 1
    rnew = 1/β - 1
    if iter == 0
        println("Iterating on prices...")
    end
    while ((error > toler_prices) && (iter < maxiter_prices))
        println("////////////////////")
        println("Price Iteration: $iter")
        r_iter = (r + rnew) / 2
        Kd = ((A^α * L ^ (1-α)) / (r_iter + δ))^1/(1-α)
        w_iter = (1-α) * A * (Kd/L)^α
        Φ = w_iter * (exp(minimum(Zvals))/r_iter)
        if ϕ > 0
            ϕ_iter = min(Φ, exp(minimum(Zvals)))
            pars = (; pars..., ϕ = ϕ_iter, r = r_iter, w = w_iter)
        else
            pars = (; pars..., r = r_iter, w = w_iter)
        end
        v_new, policy = hpi(v_init, policy, Π, Zvals, Avals, pars)
        Invariant = invariant_distribution(M, O, X, Y, Inv, policy, Π, Avals, n, pars)
        G = reshape(policy .- ϕ, n, 1)
        Ks = dot(Invariant', G)
        diff = Ks - Kd
        error = abs(diff)
        if diff > 0
            rnew = r_iter
        else
            r = r_iter
        end
        if iter % print_skip_prices == 0
            println("%%%%%%%%%%%%%%%%%%%%")
            println("Iter = $iter, Ks = $Ks, Kd = $Kd, diff = $diff")
            println("%%%%%%%%%%%%%%%%%%%%")
        end
        v_init .= v_new
        iter += 1
    end
    r_final = r + rnew / 2
    w_final = (1-α) * A * (Kd/L)^α
    Invariant = reshape(Invariant, na, nz)
    for i in 1:na
        for j in 1:nz
            wealth[i,j] = w_final * exp(Zvals[j]) + (1 + r_final) * Avals[i]
        end
    end
    println("%%%%%%%%%%%%%%%%%%%%")
    println("r = $r_final, w = $w_final")
    println("%%%%%%%%%%%%%%%%%%%%")
    return v_init, policy, Invariant, wealth
end

@time begin 
    v_hpi_crra, pol_hpi_crra, dist_hpi_crra, wealth_hpi_crra = equilibrium_hpi_crra(pars_crra)
end

Iterating on prices...
////////////////////
Price Iteration: 0
Iterating...
--------------------
Iteration: 0, Error: 0.9780765797572392
--------------------
Converged in 7 iterations
--------------------
%%%%%%%%%%%%%%%%%%%%
Iter = 0, Ks = 0.044936967336787814, Kd = 0.049144974801661453, diff = -0.004208007464873639
%%%%%%%%%%%%%%%%%%%%
////////////////////
Price Iteration: 1
Iterating...
--------------------
Iteration: 0, Error: 2.0977457894724867
--------------------
Converged in 4 iterations
--------------------
%%%%%%%%%%%%%%%%%%%%
Iter = 1, Ks = 0.05921955020310552, Kd = 0.04078698588981427, diff = 0.018432564313291253
%%%%%%%%%%%%%%%%%%%%
////////////////////
Price Iteration: 2
Iterating...
--------------------
Iteration: 0, Error: 1.0973398028501498
--------------------
Converged in 4 iterations
--------------------
%%%%%%%%%%%%%%%%%%%%
Iter = 2, Ks = 0.04713452063948005, Kd = 0.04457759796135835, diff = 0.002556922678121701
%%%%%%%%%%%%%%%%%%%%
////////////////////
Price Itera

([-42.67992818326242 -42.25566411455061 … -28.543066084318735 -25.838434771075324; -42.42590220512399 -42.082372471644334 … -28.541799906896337 -25.837487063554985; … ; 65.09242755545054 65.09242774566853 … 65.0929153630066 65.0931733329031; 65.49442587570356 65.4944260584244 … 65.49489445759855 65.49514226063604], [2.2682167995095997e-16 4.240185886515615e-16 … 0.41368701143900855 0.6484236914830955; 3.5663618627803524e-16 2.822037545510818e-16 … 0.4137820237232947 0.6485190183113092; … ; 9170.648672083138 9170.648848664292 … 9171.096077650522 9171.332688330342; 9546.82166931308 9546.821843743352 … 9547.26899660038 9547.505576922124], [8.962592246020571e-8 2.8743244642227218e-6 … 2.8729502850756924e-6 8.853853294889154e-8; 4.521479838854117e-8 2.0749242291203544e-6 … 2.073592087384129e-6 4.4127207480847414e-8; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0003667857771892585 0.0005606192817358414 … 0.497458359385241 0.7603477710318676; 0.00047598716607814736 0.0006698206706247303 … 0.

In [16]:
pars_ces = (;A = 0.01, # total factor productivity
    α = 0.33, # capital share
    δ = 0.08, # depreciation rate
    r = 0.00, # interest rate
    w = 1.0, # wage
    β = 0.9, # discount factor
    γ = 2.0, # risk aversion (recall that this must be greater than 1 in the CES formulation)
    nz = 19, # number of grid points for z
    ρ = 0.9, # persistence of AR1
    μ = 0.0, # mean of AR1
    σ = 0.003, # std of AR1
    na = 101, # number of asset grid points
    ϕ = 0.0, # borrowing constraint
    θ = 4.0, # grid expansion factor
    toler = 4e-7, # tolerance
    maxiter = 1000, # maximum no. of iterations
    print_skip = 1000, # how often to print
    how_iter = 50, # number of Howard iterations
    dampened_how = true, # whether to dampen Howard improvement step
    toler_prices = 1e-3, # tolerance
    maxiter_prices = 100, # maximum no. of iterations
    print_skip_prices = 1, # how often to print
    lb = 0.0, # lower bound of capital grid
    ub = 10000.0) # upper bound of capital grid

(A = 0.01, α = 0.33, δ = 0.08, r = 0.0, w = 1.0, β = 0.9, γ = 2.0, nz = 19, ρ = 0.9, μ = 0.0, σ = 0.003, na = 101, ϕ = 0.0, θ = 4.0, toler = 4.0e-7, maxiter = 1000, print_skip = 1000, how_iter = 50, dampened_how = true, toler_prices = 0.001, maxiter_prices = 100, print_skip_prices = 1, lb = 0.0, ub = 10000.0)

In [19]:
function equilibrium_vfi_CES(pars)
    (; na, nz, toler_prices, maxiter_prices, print_skip_prices, r, w, β, A, α, ϕ, δ) = pars
    n = na * nz
    M = zeros(na, nz, na, nz)
    O = zeros(n, n)
    X = zeros(n+1,n)
    Y = zeros(n+1)
    Inv = zeros(n)
    Π, Zvals = ar1(pars)
    Avals = exp_grid(pars)
    v_init = zeros(na,nz)
    policy = similar(v_init)
    wealth = zeros(na,nz)
    initial_distribution = zeros(n,1)
    L = labour_supply(Π, Zvals, pars)
    Ks, Kd = 1, 1
    iter = 0
    error = toler_prices + 1
    rnew = 1/β - 1
    if iter == 0
        println("Iterating on prices...")
    end
    while ((error > toler_prices) && (iter < maxiter_prices))
        println("////////////////////")
        println("Price Iteration: $iter")
        r_iter = (r + rnew) / 2
        Kd = ((A^α * L ^ (1-α)) / (r_iter + δ))^1/(1-α)
        w_iter = (1-α) * A * (Kd/L)^α
        Φ = w_iter * (exp(minimum(Zvals))/r_iter)
        if ϕ > 0
            ϕ_iter = min(Φ, exp(minimum(Zvals)))
            pars = (; pars..., ϕ = ϕ_iter, r = r_iter, w = w_iter)
        else
            pars = (; pars..., r = r_iter, w = w_iter)
        end
        v_new, policy = vfi_CES(v_init, policy, Π, Zvals, Avals, pars)
        Invariant = invariant_distribution(M, O, X, Y, Inv, policy, Π, Avals, n, pars)
        G = reshape(policy .- ϕ, n, 1)
        Ks = dot(Invariant', G)
        diff = Ks - Kd
        error = abs(diff)
        if diff > 0
            rnew = r_iter
        else
            r = r_iter
        end
        if iter % print_skip_prices == 0
            println("%%%%%%%%%%%%%%%%%%%%")
            println("Iter = $iter, Ks = $Ks, Kd = $Kd, diff = $diff")
            println("%%%%%%%%%%%%%%%%%%%%")
        end
        v_init .= v_new
        iter += 1
    end
    r_final = r + rnew / 2
    w_final = (1-α) * A * (Kd/L)^α
    Invariant = reshape(Invariant, na, nz)
    for i in 1:na
        for j in 1:nz
            wealth[i,j] = w_final * exp(Zvals[j]) + (1 + r_final) * Avals[i]
        end
    end
    println("%%%%%%%%%%%%%%%%%%%%")
    println("r = $r_final, w = $w_final")
    println("%%%%%%%%%%%%%%%%%%%%")
    return v_init, policy, Invariant, wealth
end

@time begin 
    v_vfi_ces, pol_vfi_ces, dist_vfi_ces, wealth_vfi_ces = equilibrium_vfi_CES(pars_ces)
end

Iterating on prices...
////////////////////
Price Iteration: 0
Iterating...
--------------------
Iteration: 0, Error: 0.5
--------------------
Converged in 2 iterations
--------------------
%%%%%%%%%%%%%%%%%%%%
Iter = 0, Ks = 0.016176608008028764, Kd = 0.049144974801661453, diff = -0.032968366793632686
%%%%%%%%%%%%%%%%%%%%
////////////////////
Price Iteration: 1
Iterating...
--------------------
Iteration: 0, Error: 0.0
--------------------
Converged in 1 iterations
--------------------
%%%%%%%%%%%%%%%%%%%%
Iter = 1, Ks = 0.015456700130665049, Kd = 0.04078698588981427, diff = -0.02533028575914922
%%%%%%%%%%%%%%%%%%%%
////////////////////
Price Iteration: 2
Iterating...
--------------------
Iteration: 0, Error: 0.0
--------------------
Converged in 1 iterations
--------------------
%%%%%%%%%%%%%%%%%%%%
Iter = 2, Ks = 0.015182054428997307, Kd = 0.03759051364139622, diff = -0.022408459212398913
%%%%%%%%%%%%%%%%%%%%
////////////////////
Price Iteration: 3
Iterating...
--------------------


In [27]:
function gini_coeff(distribution, wealth, pars)
    (; na, nz) = pars
    n = na * nz
    gini_w = zeros(n)
    gini_dist = zeros(n)
    lorenz = zeros(n)
    lorenz2 = zeros(n)
    wealth = reshape(wealth, n)
    distribution = reshape(distribution, n)
    sorted_wealth = sort(wealth, rev = false)
    sorted_indices = sortperm(wealth)
    sorted_distribution = distribution[sorted_indices]
    for i in 1:n
        gini_w[i] = sorted_wealth[i] * sorted_distribution[i]
        if i == 1
            lorenz[i] = gini_w[i]
            gini_dist[i] = sorted_distribution[i]
        else
            lorenz[i] = lorenz[i-1] + gini_w[i]
            gini_dist[i] = gini_dist[i-1] + sorted_distribution[i]
        end
    end
    max = lorenz[n]
    for i in 1:n
        lorenz2[i] = lorenz[i] / max
        gini_dist[i] = gini_dist[i] / gini_dist[n]
    end
    max_index = findfirst(x -> x == 1.0, lorenz2)
    if isnothing(max_index)
        max_index = n
    end
    lorenz2 = lorenz2[1:max_index]
    gini_dist = gini_dist[1:max_index]
    spl = Spline1D(gini_dist, lorenz2, k = 1)
    B = integrate(spl, 0, 1)
    Gini = 1-2*B
    Gini_round = round(Gini, digits = 4)
    fig = Figure(size = (800, 600))
    ax = Axis(fig[1, 1], xlabel = "Cumulative share of population", ylabel = "Cumulative share of wealth")
    lines!(ax, gini_dist, lorenz2, color = :blue, )
    lines!(ax, gini_dist, gini_dist, color = :red, linestyle = :dash)
    return Gini_round
end

gini_coeff(dist, wealth, pars)

NaN